In [ ]:
import random
import time
import copy
from collections import Counter
import csv
import scipy


from utils import *
from datasets import *
from mdav import *
from train import *
from models import *
from attacks import *

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset



from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics

%matplotlib inline

In [2]:
import warnings
from sklearn.exceptions import ConvergenceWarning, FitFailedWarning

# Filter out ConvergenceWarning and FitFailedWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FitFailedWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Assuming y_test and y_forget are arrays of class indices
encoder = OneHotEncoder(sparse_output=False, categories="auto")

In [3]:
# def seed_everything(seed=7):
#     np.random.seed(seed)
#     np.random.seed(seed)
#     random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
    
# seed_everything(seed=7)

In [ ]:
# Step 1: Get dataset

df=pd.read_csv('data/GiveMeSomeCredit/cs-training.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.dropna(inplace=True)
y = df['SeriousDlqin2yrs'].values
df.drop(df[['SeriousDlqin2yrs']],axis=1,inplace=True)
X = df.values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

SC = StandardScaler()
X_train = SC.fit_transform(X_train)
X_test = SC.transform(X_test)

counter = Counter(y_train)
for k,v in counter.items():
    per = v / len(y_train) * 100
    print('Class=%s, Count=%d, Percentage=%.2f%%' % (k, v, per))
    
num_features = X_train.shape[-1]
num_classes = len(set(y_train))


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
initial_model = XGBClassifier(num_classes= num_classes, reg_lambda=5, 
                              learning_rate=0.5, max_depth=9, n_estimators=200, device = device)
n_repeat = 1

In [ ]:
# Randomly sample retain and forget sets
forget_ratios = [0.05, 0.2, 0.5]
for forget_ratio in forget_ratios:
    idxs = np.arange(len(y_train))
    random.shuffle(idxs)
    m = int(len(y_train)*forget_ratio)
    retain_idxs = idxs[m:]
    forget_idxs = idxs[:m]
    X_retain = X_train[retain_idxs]
    y_retain = y_train[retain_idxs]
    X_forget = X_train[forget_idxs]
    y_forget = y_train[forget_idxs]

    # Step 2: Define and train M on D
    train_accs = []
    test_accs = []
    mia_aucs = []
    mia_advs = []
    runtimes = []

    for r in range(n_repeat):
        model = copy.deepcopy(initial_model)
        t0 = time.time()
        torch.cuda.empty_cache()
        model.fit(X_train, y_train)
        t1 = time.time()
        rt = t1-t0
        runtimes.append(rt)

        # Evaluate the model accuracy, and MIA
        # Accuracy
        train_acc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
        test_acc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
        train_accs.append(100.0*train_acc)
        test_accs.append(100.0*test_acc)
        #MIA
        idxs = np.arange(len(y_test))
        random.shuffle(idxs)
        rand_idxs = idxs[:m]

        test_preds = model.predict_proba(X_test)
        forget_preds = model.predict_proba(X_forget)

        # Convert class indices to one-hot encoding
        y_test_one_hot = encoder.fit_transform(y_test.reshape(-1, 1))
        y_forget_one_hot = encoder.transform(y_forget.reshape(-1, 1))

        loss_test = np.array([metrics.log_loss(y_test_one_hot[i], test_preds[i]) for i in range(len(y_test))])
        loss_forget = np.array([metrics.log_loss(y_forget_one_hot[i], forget_preds[i]) for i in range(len(y_forget))])

        attack_result = tf_attack(logits_train = forget_preds, logits_test = test_preds[rand_idxs], 
                                  loss_train = loss_forget, loss_test = loss_test[rand_idxs], 
                                  train_labels = y_forget, test_labels = y_test[rand_idxs])

        auc = attack_result.get_result_with_max_auc().get_auc()
        adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
        mia_aucs.append(100.0*auc)
        mia_advs.append(100.0*adv)

    mean_runtime = np.mean(runtimes)
    std_runtime = np.std(runtimes)
    mean_train_acc = np.mean(train_accs)
    std_train_acc = np.std(train_accs)
    mean_test_acc = np.mean(test_accs)
    std_test_acc = np.std(test_accs)
    mean_mia_auc = np.mean(mia_aucs)
    std_mia_auc = np.std(mia_aucs)
    mean_mia_adv = np.mean(mia_advs)
    std_mia_adv = np.std(mia_advs)

    # Print the results
    print('Training M on D time:{:0.2f}(±{:0.2f}) seconds'.format(mean_runtime, std_runtime))
    print('Train AUC:{:0.2f}(±{:0.2f})%'.format(mean_train_acc, std_train_acc))
    print('Test AUC:{:0.2f}(±{:0.2f})%'.format(mean_test_acc, std_test_acc))
    print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_mia_auc, std_mia_auc))
    print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_mia_adv, std_mia_adv))

    # Save to CSV
    csv_file_path = 'results/credit/xgb_m_d_fr={}.csv'.format(forget_ratio)

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
        writer.writerow(['Training Time', mean_runtime, std_runtime])
        writer.writerow(['Train AUC', mean_train_acc, std_train_acc])
        writer.writerow(['Test AUC', mean_test_acc, std_test_acc])
        writer.writerow(['MIA AUC', mean_mia_auc, std_mia_auc])
        writer.writerow(['MIA Advantage', mean_mia_adv, std_mia_adv])

    ######################################################################################################
    # Step 3: Train M_retain on D_retain
    retain_accs = []
    forget_accs = []
    test_accs = []
    mia_aucs = []
    mia_advs = []
    runtimes = []
    for r in range(n_repeat):
        model_ret = copy.deepcopy(initial_model)
        t0 = time.time()
        torch.cuda.empty_cache()
        model_ret.fit(X_retain, y_retain)
        t1 = time.time()
        rt = t1-t0
        runtimes.append(rt)

        # Evaluate the model accuracy, and MIA
        # Accuracy
        retain_acc = roc_auc_score(y_retain, model_ret.predict_proba(X_retain)[:, 1])
        forget_acc = roc_auc_score(y_forget, model_ret.predict_proba(X_forget)[:, 1])
        test_acc = roc_auc_score(y_test, model_ret.predict_proba(X_test)[:, 1])
        retain_accs.append(100.0*retain_acc)
        forget_accs.append(100.0*forget_acc)
        test_accs.append(100.0*test_acc)
        #MIA
        idxs = np.arange(len(y_test))
        random.shuffle(idxs)
        rand_idxs = idxs[:m]

        test_preds = model_ret.predict_proba(X_test)
        forget_preds = model_ret.predict_proba(X_forget)
        loss_test = np.array([metrics.log_loss(y_test_one_hot[i], test_preds[i]) for i in range(len(y_test))])
        loss_forget = np.array([metrics.log_loss(y_forget_one_hot[i], forget_preds[i]) for i in range(len(y_forget))])

        attack_result = tf_attack(logits_train = forget_preds, logits_test = test_preds[rand_idxs], 
                                  loss_train = loss_forget, loss_test = loss_test[rand_idxs], 
                                  train_labels = y_forget, test_labels = y_test[rand_idxs])

        auc = attack_result.get_result_with_max_auc().get_auc()
        adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
        mia_aucs.append(100.0*auc)
        mia_advs.append(100.0*adv)


    mean_retrain_runtime = np.mean(runtimes)
    std_retrain_runtime = np.std(runtimes)
    mean_retain_acc = np.mean(retain_accs)
    std_retain_acc = np.std(retain_accs)
    mean_forget_acc = np.mean(forget_accs)
    std_forget_acc = np.std(forget_accs)
    mean_retrain_test_acc = np.mean(test_accs)
    std_retrain_test_acc = np.std(test_accs)
    mean_retrain_mia_auc = np.mean(mia_aucs)
    std_retrain_mia_auc = np.std(mia_aucs)
    mean_retrain_mia_adv = np.mean(mia_advs)
    std_retrain_mia_adv = np.std(mia_advs)

    # Print the results
    print('Retraining M on D_ret time:{:0.2f}(±{:0.2f}) seconds'.format(mean_retrain_runtime, std_retrain_runtime))
    print('Retain AUC:{:0.2f}(±{:0.2f})%'.format(mean_retain_acc, std_retain_acc))
    print('Forget AUC:{:0.2f}(±{:0.2f})%'.format(mean_forget_acc, std_forget_acc))
    print('Test AUC:{:0.2f}(±{:0.2f})%'.format(mean_retrain_test_acc, std_retrain_test_acc))
    print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_retrain_mia_auc, std_retrain_mia_auc))
    print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_retrain_mia_adv, std_retrain_mia_adv))

    # Save to CSV
    csv_retrain_file_path = 'results/credit/xgb_mret_dret_fr={}.csv'.format(forget_ratio)

    with open(csv_retrain_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
        writer.writerow(['Retraining Time', mean_retrain_runtime, std_retrain_runtime])
        writer.writerow(['Retain AUC', mean_retain_acc, std_retain_acc])
        writer.writerow(['Forget AUC', mean_forget_acc, std_forget_acc])
        writer.writerow(['Test AUC', mean_retrain_test_acc, std_retrain_test_acc])
        writer.writerow(['MIA AUC', mean_retrain_mia_auc, std_retrain_mia_auc])
        writer.writerow(['MIA Advantage', mean_retrain_mia_adv, std_retrain_mia_adv])
        
# ######################################################################################################
    # Step 1: k-anonymize and prepare D_k
    ft_epochs_list = [5]
    for ft_epochs in ft_epochs_list:
        K = [10]
        for k in K:
            runtimes_k = []
            t0 = time.time()
            centroids, clusters, labels, X_train_k, y_train_k = mdav(copy.deepcopy(X_train), copy.deepcopy(y_train), k)
            print_stats(clusters, centroids)
            print('Shape of X_train_k:{}, y_train_k:{}'.format(X_train_k.shape, y_train_k.shape))
             # Create TensorDatasets
            t1 = time.time()
            rt_k = t1- t0
            runtimes_k.append(rt_k)

            train_accs_k = []
            test_accs_k = []
            mia_aucs_k = []
            mia_advs_k = []
            runtimes_train_k = []

            train_accs_k_D = []
            test_accs_k_D = []
            mia_aucs_k_D = []
            mia_advs_k_D = []
            runtimes_train_k_D = []

            retain_accs_k_ret = []
            forget_accs_k_ret = []
            test_accs_k_ret = []
            mia_aucs_k_ret = []
            mia_advs_k_ret = []
            runtimes_train_k_ret = []

            for r in range(n_repeat):
                model_k = copy.deepcopy(initial_model)
                t0 = time.time()
                torch.cuda.empty_cache()
                model_k.fit(X_train_k, y_train_k)
                t1 = time.time()
                rt_train = t1- t0
                runtimes_train_k.append(rt_train)

                # Evaluate the model accuracy, and MIA
                # Accuracy
                train_acc = roc_auc_score(y_train_k, model_k.predict_proba(X_train_k)[:, 1])
                test_acc = roc_auc_score(y_test, model_k.predict_proba(X_test)[:, 1])
                train_accs_k.append(100.0*train_acc)
                test_accs_k.append(100.0*test_acc)
                #MIA
                idxs = np.arange(len(y_test))
                random.shuffle(idxs)
                rand_idxs = idxs[:m]

                test_preds = model_k.predict_proba(X_test)
                forget_preds = model_k.predict_proba(X_forget)

                # Convert class indices to one-hot encoding
                y_test_one_hot = encoder.fit_transform(y_test.reshape(-1, 1))
                y_forget_one_hot = encoder.transform(y_forget.reshape(-1, 1))

                loss_test = np.array([metrics.log_loss(y_test_one_hot[i], test_preds[i]) for i in range(len(y_test))])
                loss_forget = np.array([metrics.log_loss(y_forget_one_hot[i], forget_preds[i]) for i in range(len(y_forget))])

                attack_result = tf_attack(logits_train = forget_preds, logits_test = test_preds[rand_idxs], 
                                          loss_train = loss_forget, loss_test = loss_test[rand_idxs], 
                                          train_labels = y_forget, test_labels = y_test[rand_idxs])

                auc = attack_result.get_result_with_max_auc().get_auc()
                adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
                mia_aucs_k.append(100.0*auc)
                mia_advs_k.append(100.0*adv)

                model_k_D = copy.deepcopy(initial_model)
                model_k_D.set_params(learning_rate = 0.5, n_estimators=ft_epochs)
                t0 = time.time()
                torch.cuda.empty_cache()
                model_k_D.fit(X_train, y_train, xgb_model=model_k)
                t1 = time.time()
                rt = t1-t0
                runtimes_train_k_D.append(rt)

                # Evaluate the model accuracy, and MIA
                # Accuracy
                train_acc = roc_auc_score(y_train, model_k_D.predict_proba(X_train)[:, 1])
                test_acc = roc_auc_score(y_test, model_k_D.predict_proba(X_test)[:, 1])
                train_accs_k_D.append(100.0*train_acc)
                test_accs_k_D.append(100.0*test_acc)
                #MIA
                idxs = np.arange(len(y_test))
                random.shuffle(idxs)
                rand_idxs = idxs[:m]

                test_preds = model_k_D.predict_proba(X_test)
                forget_preds = model_k_D.predict_proba(X_forget)

                # Convert class indices to one-hot encoding
                y_test_one_hot = encoder.fit_transform(y_test.reshape(-1, 1))
                y_forget_one_hot = encoder.transform(y_forget.reshape(-1, 1))

                loss_test = np.array([metrics.log_loss(y_test_one_hot[i], test_preds[i]) for i in range(len(y_test))])
                loss_forget = np.array([metrics.log_loss(y_forget_one_hot[i], forget_preds[i]) for i in range(len(y_forget))])

                attack_result = tf_attack(logits_train = forget_preds, logits_test = test_preds[rand_idxs], 
                                          loss_train = loss_forget, loss_test = loss_test[rand_idxs], 
                                          train_labels = y_forget, test_labels = y_test[rand_idxs])

                auc = attack_result.get_result_with_max_auc().get_auc()
                adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
                mia_aucs_k_D.append(100.0*auc)
                mia_advs_k_D.append(100.0*adv)

                model_k_ret = copy.deepcopy(initial_model)
                model_k_ret.set_params(learning_rate = 0.5, n_estimators=ft_epochs)
                t0 = time.time()
                torch.cuda.empty_cache()
                model_k_ret.fit(X_retain, y_retain, xgb_model=model_k)
                t1 = time.time()
                rt = t1-t0
                runtimes_train_k_ret.append(rt)
                # Evaluate the model accuracy, and MIA
                # Accuracy
                retain_acc = roc_auc_score(y_retain, model_k_ret.predict_proba(X_retain)[:, 1])
                forget_acc = roc_auc_score(y_forget, model_k_ret.predict_proba(X_forget)[:, 1])
                test_acc = roc_auc_score(y_test, model_k_ret.predict_proba(X_test)[:, 1])
                retain_accs_k_ret.append(100.0*retain_acc)
                forget_accs_k_ret.append(100.0*forget_acc)
                test_accs_k_ret.append(100.0*test_acc)
                #MIA
                idxs = np.arange(len(y_test))
                random.shuffle(idxs)
                rand_idxs = idxs[:m]

                test_preds = model_k_ret.predict_proba(X_test)
                forget_preds = model_k_ret.predict_proba(X_forget)
                loss_test = np.array([metrics.log_loss(y_test_one_hot[i], test_preds[i]) for i in range(len(y_test))])
                loss_forget = np.array([metrics.log_loss(y_forget_one_hot[i], forget_preds[i]) for i in range(len(y_forget))])

                attack_result = tf_attack(logits_train = forget_preds, logits_test = test_preds[rand_idxs], 
                                          loss_train = loss_forget, loss_test = loss_test[rand_idxs], 
                                          train_labels = y_forget, test_labels = y_test[rand_idxs])

                auc = attack_result.get_result_with_max_auc().get_auc()
                adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
                mia_aucs_k_ret.append(100.0*auc)
                mia_advs_k_ret.append(100.0*adv)


            # Anonymizing D and training M_k on D_k
            mean_anonymize_time = np.mean(runtimes_k)
            std_anonymize_time = np.std(runtimes_k)
            mean_train_k_time = np.mean(runtimes_train_k)
            std_train_k_time = np.std(runtimes_train_k)
            mean_train_k_acc = np.mean(train_accs_k)
            std_train_k_acc = np.std(train_accs_k)
            mean_test_k_acc = np.mean(test_accs_k)
            std_test_k_acc = np.std(test_accs_k)
            mean_mia_k_auc = np.mean(mia_aucs_k)
            std_mia_k_auc = np.std(mia_aucs_k)
            mean_mia_k_adv = np.mean(mia_advs_k)
            std_mia_k_adv = np.std(mia_advs_k)

            # Finetuning M_k on D
            mean_finetune_D_time = np.mean(runtimes_train_k_D)
            std_finetune_D_time = np.std(runtimes_train_k_D)
            mean_finetune_D_train_acc = np.mean(train_accs_k_D)
            std_finetune_D_train_acc = np.std(train_accs_k_D)
            mean_finetune_D_test_acc = np.mean(test_accs_k_D)
            std_finetune_D_test_acc = np.std(test_accs_k_D)
            mean_finetune_D_mia_auc = np.mean(mia_aucs_k_D)
            std_finetune_D_mia_auc = np.std(mia_aucs_k_D)
            mean_finetune_D_mia_adv = np.mean(mia_advs_k_D)
            std_finetune_D_mia_adv = np.std(mia_advs_k_D)

            # Finetuning M_k on D_ret
            mean_finetune_D_ret_time = np.mean(runtimes_train_k_ret)
            std_finetune_D_ret_time = np.std(runtimes_train_k_ret)
            mean_finetune_D_ret_train_acc = np.mean(retain_accs_k_ret)
            std_finetune_D_ret_train_acc = np.std(retain_accs_k_ret)
            mean_finetune_D_ret_forget_acc = np.mean(forget_accs_k_ret)
            std_finetune_D_ret_forget_acc = np.std(forget_accs_k_ret)
            mean_finetune_D_ret_test_acc = np.mean(test_accs_k_ret)
            std_finetune_D_ret_test_acc = np.std(test_accs_k_ret)
            mean_finetune_D_ret_mia_auc = np.mean(mia_aucs_k_ret)
            std_finetune_D_ret_mia_auc = np.std(mia_aucs_k_ret)
            mean_finetune_D_ret_mia_adv = np.mean(mia_advs_k_ret)
            std_finetune_D_ret_mia_adv = np.std(mia_advs_k_ret)


            # Print the results
            print('----------------------------------------')
            print('k=', k, 'Fine-tuning epochs=', ft_epochs)
            print('----------------------------------------')
            print('-----Anonymizing D and training M_k on D_k-----')
            print('Anonymizing D time:{:0.2f}(±{:0.2f})'.format(mean_anonymize_time, std_anonymize_time))
            print('Training M_k on D_k time:{:0.2f}(±{:0.2f})'.format(mean_train_k_time, std_train_k_time))
            print('Train AUC:{:0.2f}(±{:0.2f})%'.format(mean_train_k_acc, std_train_k_acc))
            print('Test AUC:{:0.2f}(±{:0.2f})%'.format(mean_test_k_acc, std_test_k_acc))
            print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_mia_k_auc, std_mia_k_auc))
            print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_mia_k_adv, std_mia_k_adv))

            print('-----Finetuning M_k on D-----')
            print('Training M_k on D time:{:0.2f}(±{:0.2f})'.format(mean_finetune_D_time, std_finetune_D_time))
            print('Train AUC:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_train_acc, std_finetune_D_train_acc))
            print('Test AUC:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_test_acc, std_finetune_D_test_acc))
            print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_mia_auc, std_finetune_D_mia_auc))
            print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_mia_adv, std_finetune_D_mia_adv))

            print('-----Finetuning M_k on D_ret-----')
            print('Finetuning M_k on D_retain time:{:0.2f}(±{:0.2f}) seconds'.format(mean_finetune_D_ret_time, std_finetune_D_ret_time))
            print('Retain AUC:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_train_acc, std_finetune_D_ret_train_acc))
            print('Forget AUC:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_forget_acc, std_finetune_D_ret_forget_acc))
            print('Test AUC:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_test_acc, std_finetune_D_ret_test_acc))
            print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_mia_auc, std_finetune_D_ret_mia_auc))
            print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_mia_adv, std_finetune_D_ret_mia_adv))
            print('----------------------------------------')

            # Save to CSV
            csv_anonymize_file_path = 'results/credit/xgb_mk={}_dk_fr={}.csv'.format(k, forget_ratio)
            csv_finetune_D_file_path = 'results/credit/xgb_mk={}_d_fr={}_epochs={}.csv'.format(k, forget_ratio, ft_epochs)
            csv_finetune_D_ret_file_path = 'results/credit/xgb_mk={}_dret_fr={}_epochs={}.csv'.format(k, forget_ratio, ft_epochs)

            # Writing to CSV for anonymizing, finetuning on D, and finetuning on D_ret
            with open(csv_anonymize_file_path, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
                writer.writerow(['Anonymizing Time', mean_anonymize_time, std_anonymize_time])
                writer.writerow(['Training M_k on D_k Time', mean_train_k_time, std_train_k_time])
                writer.writerow(['Train AUC', mean_train_k_acc, std_train_k_acc])
                writer.writerow(['Test AUC', mean_test_k_acc, std_test_k_acc])
                writer.writerow(['MIA AUC', mean_mia_k_auc, std_mia_k_auc])
                writer.writerow(['MIA Advantage', mean_mia_k_adv, std_mia_k_adv])

            with open(csv_finetune_D_file_path, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
                writer.writerow(['Training M_k on D Time', mean_finetune_D_time, std_finetune_D_time])
                writer.writerow(['Train AUC', mean_finetune_D_train_acc, std_finetune_D_train_acc])
                writer.writerow(['Test AUC', mean_finetune_D_test_acc, std_finetune_D_test_acc])
                writer.writerow(['MIA AUC', mean_finetune_D_mia_auc, std_finetune_D_mia_auc])
                writer.writerow(['MIA Advantage', mean_finetune_D_mia_adv, std_finetune_D_mia_adv])

            with open(csv_finetune_D_ret_file_path, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
                writer.writerow(['Finetuning M_k on D_retain Time', mean_finetune_D_ret_time, std_finetune_D_ret_time])
                writer.writerow(['Retain AUC', mean_finetune_D_ret_train_acc, std_finetune_D_ret_train_acc])
                writer.writerow(['Forget AUC', mean_finetune_D_ret_forget_acc, std_finetune_D_ret_forget_acc])
                writer.writerow(['Test AUC', mean_finetune_D_ret_test_acc, std_finetune_D_ret_test_acc])
                writer.writerow(['MIA AUC', mean_finetune_D_ret_mia_auc, std_finetune_D_ret_mia_auc])
                writer.writerow(['MIA Advantage', mean_finetune_D_ret_mia_adv, std_finetune_D_ret_mia_adv])

# Differential privacy

In [ ]:
# Randomly sample retain and forget sets
forget_ratios = [0.05, 0.2, 0.5]
# Step 1: k-anonymize and prepare D_k
ft_epochs_list = [5]
m = int(len(y_train)*forget_ratio)
idxs = np.arange(len(y_train))
random.shuffle(idxs)

retain_idxs = idxs[m:]
forget_idxs = idxs[:m]
X_retain = X_train[retain_idxs]
y_retain = y_train[retain_idxs]
X_forget = X_train[forget_idxs]
y_forget = y_train[forget_idxs]

for forget_ratio in forget_ratios:
    idxs = np.arange(len(y_train))
    random.shuffle(idxs)
    m = int(len(y_train)*forget_ratio)
    retain_idxs = idxs[m:]
    forget_idxs = idxs[:m]
    X_retain = X_train[retain_idxs]
    y_retain = y_train[retain_idxs]
    X_forget = X_train[forget_idxs]
    y_forget = y_train[forget_idxs]

    for r in range(n_repeat):
        ft_epochs_list = [5]
        for ft_epochs in ft_epochs_list:
            EPS = [0.5]
            for eps in EPS:
                dp_train_data = pd.read_csv('dp_data/GiveMeSomeCredit/dp_credit_eps={}.csv'.format(eps), sep=',')
                # Drop useless columns
                dp_train_data.dropna(inplace=True)
                # convert the income column to 0 or 1 and then drop the column for the feature vectors
                # creating the feature vector 
                X_train_dp = dp_train_data.drop('SeriousDlqin2yrs', axis =1)
                # target values
                y_train_dp = dp_train_data['SeriousDlqin2yrs'].values
                # pass the data through the full_pipeline
                X_train_dp = SC.fit_transform(X_train_dp)
                print(forget_ratio, ft_epochs, eps, X_train_dp.shape, y_train_dp.shape)
                train_accs_k = []
                test_accs_k = []
                mia_aucs_k = []
                mia_advs_k = []
                runtimes_train_k = []

                train_accs_k_D = []
                test_accs_k_D = []
                mia_aucs_k_D = []
                mia_advs_k_D = []
                runtimes_train_k_D = []

                retain_accs_k_ret = []
                forget_accs_k_ret = []
                test_accs_k_ret = []
                mia_aucs_k_ret = []
                mia_advs_k_ret = []
                runtimes_train_k_ret = []

                for r in range(n_repeat):
                    model_k = copy.deepcopy(initial_model)
                    t0 = time.time()
                    torch.cuda.empty_cache()
                    model_k.fit(X_train_dp, y_train_dp)
                    t1 = time.time()
                    rt_train = t1- t0
                    runtimes_train_k.append(rt_train)

                    # Evaluate the model accuracy, and MIA
                    # Accuracy
                    train_acc = roc_auc_score(y_train_dp, model_k.predict_proba(X_train_dp)[:, 1])
                    test_acc = roc_auc_score(y_test, model_k.predict_proba(X_test)[:, 1])
                    train_accs_k.append(100.0*train_acc)
                    test_accs_k.append(100.0*test_acc)
                    #MIA
                    idxs = np.arange(len(y_test))
                    random.shuffle(idxs)
                    rand_idxs = idxs[:m]

                    test_preds = model_k.predict_proba(X_test)
                    forget_preds = model_k.predict_proba(X_forget)

                    # Convert class indices to one-hot encoding
                    y_test_one_hot = encoder.fit_transform(y_test.reshape(-1, 1))
                    y_forget_one_hot = encoder.transform(y_forget.reshape(-1, 1))

                    loss_test = np.array([metrics.log_loss(y_test_one_hot[i], test_preds[i]) for i in range(len(y_test))])
                    loss_forget = np.array([metrics.log_loss(y_forget_one_hot[i], forget_preds[i]) for i in range(len(y_forget))])

                    attack_result = tf_attack(logits_train = forget_preds, logits_test = test_preds[rand_idxs], 
                                              loss_train = loss_forget, loss_test = loss_test[rand_idxs], 
                                              train_labels = y_forget, test_labels = y_test[rand_idxs])

                    auc = attack_result.get_result_with_max_auc().get_auc()
                    adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
                    mia_aucs_k.append(100.0*auc)
                    mia_advs_k.append(100.0*adv)

                    model_k_D = copy.deepcopy(initial_model)
                    model_k_D.set_params(learning_rate = 0.5, n_estimators=ft_epochs)
                    t0 = time.time()
                    torch.cuda.empty_cache()
                    model_k_D.fit(X_train, y_train, xgb_model=model_k)
                    t1 = time.time()
                    rt = t1-t0
                    runtimes_train_k_D.append(rt)

                    # Evaluate the model accuracy, and MIA
                    # Accuracy
                    train_acc = roc_auc_score(y_train, model_k_D.predict_proba(X_train)[:, 1])
                    test_acc = roc_auc_score(y_test, model_k_D.predict_proba(X_test)[:, 1])
                    train_accs_k_D.append(100.0*train_acc)
                    test_accs_k_D.append(100.0*test_acc)
                    #MIA
                    idxs = np.arange(len(y_test))
                    random.shuffle(idxs)
                    rand_idxs = idxs[:m]

                    test_preds = model_k_D.predict_proba(X_test)
                    forget_preds = model_k_D.predict_proba(X_forget)

                    # Convert class indices to one-hot encoding
                    y_test_one_hot = encoder.fit_transform(y_test.reshape(-1, 1))
                    y_forget_one_hot = encoder.transform(y_forget.reshape(-1, 1))

                    loss_test = np.array([metrics.log_loss(y_test_one_hot[i], test_preds[i]) for i in range(len(y_test))])
                    loss_forget = np.array([metrics.log_loss(y_forget_one_hot[i], forget_preds[i]) for i in range(len(y_forget))])

                    attack_result = tf_attack(logits_train = forget_preds, logits_test = test_preds[rand_idxs], 
                                              loss_train = loss_forget, loss_test = loss_test[rand_idxs], 
                                              train_labels = y_forget, test_labels = y_test[rand_idxs])

                    auc = attack_result.get_result_with_max_auc().get_auc()
                    adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
                    mia_aucs_k_D.append(100.0*auc)
                    mia_advs_k_D.append(100.0*adv)

                    model_k_ret = copy.deepcopy(initial_model)
                    model_k_ret.set_params(learning_rate = 0.5, n_estimators=ft_epochs)
                    t0 = time.time()
                    torch.cuda.empty_cache()
                    model_k_ret.fit(X_retain, y_retain, xgb_model=model_k)
                    t1 = time.time()
                    rt = t1-t0
                    runtimes_train_k_ret.append(rt)
                    # Evaluate the model accuracy, and MIA
                    # Accuracy
                    retain_acc = roc_auc_score(y_retain, model_k_ret.predict_proba(X_retain)[:, 1])
                    forget_acc = roc_auc_score(y_forget, model_k_ret.predict_proba(X_forget)[:, 1])
                    test_acc = roc_auc_score(y_test, model_k_ret.predict_proba(X_test)[:, 1])
                    retain_accs_k_ret.append(100.0*retain_acc)
                    forget_accs_k_ret.append(100.0*forget_acc)
                    test_accs_k_ret.append(100.0*test_acc)
                    #MIA
                    idxs = np.arange(len(y_test))
                    random.shuffle(idxs)
                    rand_idxs = idxs[:m]

                    test_preds = model_k_ret.predict_proba(X_test)
                    forget_preds = model_k_ret.predict_proba(X_forget)
                    loss_test = np.array([metrics.log_loss(y_test_one_hot[i], test_preds[i]) for i in range(len(y_test))])
                    loss_forget = np.array([metrics.log_loss(y_forget_one_hot[i], forget_preds[i]) for i in range(len(y_forget))])

                    attack_result = tf_attack(logits_train = forget_preds, logits_test = test_preds[rand_idxs], 
                                              loss_train = loss_forget, loss_test = loss_test[rand_idxs], 
                                              train_labels = y_forget, test_labels = y_test[rand_idxs])

                    auc = attack_result.get_result_with_max_auc().get_auc()
                    adv = attack_result.get_result_with_max_attacker_advantage().get_attacker_advantage()
                    mia_aucs_k_ret.append(100.0*auc)
                    mia_advs_k_ret.append(100.0*adv)


                # Anonymizing D and training M_k on D_k
                mean_train_k_time = np.mean(runtimes_train_k)
                std_train_k_time = np.std(runtimes_train_k)
                mean_train_k_acc = np.mean(train_accs_k)
                std_train_k_acc = np.std(train_accs_k)
                mean_test_k_acc = np.mean(test_accs_k)
                std_test_k_acc = np.std(test_accs_k)
                mean_mia_k_auc = np.mean(mia_aucs_k)
                std_mia_k_auc = np.std(mia_aucs_k)
                mean_mia_k_adv = np.mean(mia_advs_k)
                std_mia_k_adv = np.std(mia_advs_k)

                # Finetuning M_k on D
                mean_finetune_D_time = np.mean(runtimes_train_k_D)
                std_finetune_D_time = np.std(runtimes_train_k_D)
                mean_finetune_D_train_acc = np.mean(train_accs_k_D)
                std_finetune_D_train_acc = np.std(train_accs_k_D)
                mean_finetune_D_test_acc = np.mean(test_accs_k_D)
                std_finetune_D_test_acc = np.std(test_accs_k_D)
                mean_finetune_D_mia_auc = np.mean(mia_aucs_k_D)
                std_finetune_D_mia_auc = np.std(mia_aucs_k_D)
                mean_finetune_D_mia_adv = np.mean(mia_advs_k_D)
                std_finetune_D_mia_adv = np.std(mia_advs_k_D)

                # Finetuning M_k on D_ret
                mean_finetune_D_ret_time = np.mean(runtimes_train_k_ret)
                std_finetune_D_ret_time = np.std(runtimes_train_k_ret)
                mean_finetune_D_ret_train_acc = np.mean(retain_accs_k_ret)
                std_finetune_D_ret_train_acc = np.std(retain_accs_k_ret)
                mean_finetune_D_ret_forget_acc = np.mean(forget_accs_k_ret)
                std_finetune_D_ret_forget_acc = np.std(forget_accs_k_ret)
                mean_finetune_D_ret_test_acc = np.mean(test_accs_k_ret)
                std_finetune_D_ret_test_acc = np.std(test_accs_k_ret)
                mean_finetune_D_ret_mia_auc = np.mean(mia_aucs_k_ret)
                std_finetune_D_ret_mia_auc = np.std(mia_aucs_k_ret)
                mean_finetune_D_ret_mia_adv = np.mean(mia_advs_k_ret)
                std_finetune_D_ret_mia_adv = np.std(mia_advs_k_ret)


                # Print the results
                print('----------------------------------------')
                print('Epsilon=', eps, 'Fine-tuning epochs=', ft_epochs)
                print('----------------------------------------')
                print('-----Anonymizing D and training M_k on D_k-----')
                print('Training M_k on D_k time:{:0.2f}(±{:0.2f})'.format(mean_train_k_time, std_train_k_time))
                print('Train accuracy:{:0.2f}(±{:0.2f})%'.format(mean_train_k_acc, std_train_k_acc))
                print('Test accuracy:{:0.2f}(±{:0.2f})%'.format(mean_test_k_acc, std_test_k_acc))
                print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_mia_k_auc, std_mia_k_auc))
                print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_mia_k_adv, std_mia_k_adv))

                print('-----Finetuning M_k on D-----')
                print('Training M_k on D time:{:0.2f}(±{:0.2f})'.format(mean_finetune_D_time, std_finetune_D_time))
                print('Train accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_train_acc, std_finetune_D_train_acc))
                print('Test accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_test_acc, std_finetune_D_test_acc))
                print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_mia_auc, std_finetune_D_mia_auc))
                print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_mia_adv, std_finetune_D_mia_adv))

                print('-----Finetuning M_k on D_ret-----')
                print('Finetuning M_k on D_retain time:{:0.2f}(±{:0.2f}) seconds'.format(mean_finetune_D_ret_time, std_finetune_D_ret_time))
                print('Retain accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_train_acc, std_finetune_D_ret_train_acc))
                print('Forget accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_forget_acc, std_finetune_D_ret_forget_acc))
                print('Test accuracy:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_test_acc, std_finetune_D_ret_test_acc))
                print('MIA AUC:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_mia_auc, std_finetune_D_ret_mia_auc))
                print('MIA Advantage:{:0.2f}(±{:0.2f})%'.format(mean_finetune_D_ret_mia_adv, std_finetune_D_ret_mia_adv))
                print('----------------------------------------')

                # Save to CSV
                csv_anonymize_file_path = 'results/credit/xgb_mdp_eps={}_fr={}.csv'.format(eps, forget_ratio)
                csv_finetune_D_file_path = 'results/credit/xgb_mdpd_eps={}_fr={}_epochs={}.csv'.format(eps, forget_ratio, ft_epochs)
                csv_finetune_D_ret_file_path = 'results/credit/xgb_mdpret_eps={}_fr={}_epochs={}.csv'.format(eps, forget_ratio, ft_epochs)

                # Writing to CSV for anonymizing, finetuning on D, and finetuning on D_ret
                with open(csv_anonymize_file_path, mode='w', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
                    writer.writerow(['Training M_k on D_k Time', mean_train_k_time, std_train_k_time])
                    writer.writerow(['Train AUC', mean_train_k_acc, std_train_k_acc])
                    writer.writerow(['Test AUC', mean_test_k_acc, std_test_k_acc])
                    writer.writerow(['MIA AUC', mean_mia_k_auc, std_mia_k_auc])
                    writer.writerow(['MIA Advantage', mean_mia_k_adv, std_mia_k_adv])

                with open(csv_finetune_D_file_path, mode='w', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
                    writer.writerow(['Training M_k on D Time', mean_finetune_D_time, std_finetune_D_time])
                    writer.writerow(['Train AUC', mean_finetune_D_train_acc, std_finetune_D_train_acc])
                    writer.writerow(['Test AUC', mean_finetune_D_test_acc, std_finetune_D_test_acc])
                    writer.writerow(['MIA AUC', mean_finetune_D_mia_auc, std_finetune_D_mia_auc])
                    writer.writerow(['MIA Advantage', mean_finetune_D_mia_adv, std_finetune_D_mia_adv])

                with open(csv_finetune_D_ret_file_path, mode='w', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(['Metric', 'Mean', 'Standard Deviation'])
                    writer.writerow(['Finetuning M_k on D_retain Time', mean_finetune_D_ret_time, std_finetune_D_ret_time])
                    writer.writerow(['Retain AUC', mean_finetune_D_ret_train_acc, std_finetune_D_ret_train_acc])
                    writer.writerow(['Forget AUC', mean_finetune_D_ret_forget_acc, std_finetune_D_ret_forget_acc])
                    writer.writerow(['Test AUC', mean_finetune_D_ret_test_acc, std_finetune_D_ret_test_acc])
                    writer.writerow(['MIA AUC', mean_finetune_D_ret_mia_auc, std_finetune_D_ret_mia_auc])
                    writer.writerow(['MIA Advantage', mean_finetune_D_ret_mia_adv, std_finetune_D_ret_mia_adv])